In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd '/content/drive/My Drive/UF/Intro to Data Science/Project'

/content/drive/My Drive/UF/Intro to Data Science/Project


In [19]:
%ls

 11142019_train_subsampled.csv   IDS_featuer_selection_lab.ipynb
'Data Cleaning.ipynb'            IDS_featuer_selection.py


In [0]:
import pandas as pd

In [32]:
df = pd.read_csv("11142019_train_subsampled.csv")
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,channelGrouping,date,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,socialEngagementType,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime,new.returningCustomer
0,Referral,20171016,Chrome,desktop,False,Chrome OS,8934116514970143966,Cupertino,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,San Francisco-Oakland-San Jose CA,(not set),not available in demo dataset,California,Northern America,Not Socially Engaged,NaN,2,NaN,2.0,2.0,28.0,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,1508176307,6,1508176307,1
1,Direct,20171016,Chrome,mobile,True,Android,7992466427990357681,not available in demo dataset,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,Northern America,Not Socially Engaged,NaN,2,1.0,2.0,1.0,38.0,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,True,NaN,(none),NaN,(direct),1508201613,1,1508201613,0
2,Organic Search,20171016,Chrome,desktop,False,Windows,9075655783635761930,not available in demo dataset,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,Western Asia,Not Socially Engaged,NaN,2,1.0,2.0,1.0,1.0,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google,1508169851,1,1508169851,0
3,Organic Search,20171016,Chrome,desktop,False,Windows,6960673291025684308,not available in demo dataset,not available in demo dataset,Americas,Mexico,not available in demo dataset,not available in demo dataset,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,not available in demo dataset,Central America,Not Socially Engaged,NaN,2,1.0,2.0,1.0,52.0,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google,1508190552,1,1508190552,0
4,Referral,20171016,Chrome,desktop,False,Macintosh,166277907528479249,San Francisco,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,San Francisco-Oakland-San Jose CA,unknown.unknown,not available in demo dataset,California,Northern America,Not Socially Engaged,NaN,2,1.0,2.0,2.0,12.0,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,NaN,(none),/offer/2145,(direct),1508196701,1,1508196701,1


In [34]:
df.shape

(1537503, 47)

In [0]:
df['fullVisitorId'] = df['fullVisitorId'].astype(float)

In [23]:
df['totals.bounces'].value_counts()

1.0    784213
Name: totals.bounces, dtype: int64

In [31]:
# na_columns = pd.DataFrame(index=df.columns, columns=['NA count'])

# for column in df.columns:
#   # print(f"{column}: \t\t\t\t\t\t {df[column].isna().sum(axis=0)}")
#   na_columns.loc[column] = df[column].isna().sum(axis=0)

# na_columns[na_columns['NA count'] > 0]
na_columns = df.isna().sum(axis=0)
na_columns[na_columns>0]

totals.bounces                                   753290
totals.pageviews                                    221
trafficSource.adContent                         1479250
trafficSource.adwordsClickInfo.adNetworkType    1469786
trafficSource.adwordsClickInfo.gclId            1469647
trafficSource.adwordsClickInfo.slot             1469786
trafficSource.isTrueDirect                      1056703
trafficSource.keyword                            947273
trafficSource.referralPath                      1027971
dtype: int64

In [25]:
(752034)/len(df)

0.4891268504841942

In [0]:
# Columns to be dropped
# 100                   100                    100                     77                  100                           99.99                         
# 'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro', 'geoNetwork.networkLocation', 'trafficSource.campaignCode', 'trafficSource.adwordsClickInfo.isVideoAd'

In [0]:
# Set the na values of following columns
# 'geoNetwork.networkDomain': '(not set)' => 'unknown.unknown'
# 'totals.bounces': NA => -1
# 'trafficSource.referralPath' => 'UNK'
# 'totals.transactions' => 0
# 'totals.transactionRevenue' => 0
# 'totals.totalTransactionRevenue' => 0
# 'trafficSource.isTrueDirect' => False
# 'trafficSource.keyword' => 'UNK'
# 'trafficSource.adwordsClickInfo.slot' => 'UNK'
# 'trafficSource.adwordsClickInfo.page' => 0
# 'trafficSource.adwordsClickInfo.gclId' => 'UNK'
# 'trafficSource.adwordsClickInfo.adNetworkType' => 'UNK'
# 'trafficSource.adContent' => 'UNK'
# 'totals.timeOnSite' => 0 or -1
# 'totals.sessionQualityDim' => 0
# 'totals.pageviews' => 3.697734703196941
# 'totals.newVisits' => 0

In [0]:
df = df.drop(columns=['geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro', 'geoNetwork.networkLocation', 'trafficSource.campaignCode', 'trafficSource.adwordsClickInfo.isVideoAd'])

In [0]:
df['geoNetwork.networkDomain'] = df['geoNetwork.networkDomain'].map(lambda value: 'unknown.unknown' if value == '(not set)' else value)

In [0]:
fill_0_columns = ['totals.transactions', 'totals.transactionRevenue', 'totals.totalTransactionRevenue', 'trafficSource.adwordsClickInfo.page', 'totals.timeOnSite', 'totals.sessionQualityDim', 'totals.newVisits']
df[fill_0_columns] = df[fill_0_columns].fillna(0)

In [0]:
df = df.fillna({
    'totals.bounces': -1, 
    'trafficSource.referralPath': 'UNK', 
    'trafficSource.isTrueDirect': False, 
    'trafficSource.keyword': 'UNK', 
    'trafficSource.adwordsClickInfo.slot': 'UNK', 
    'trafficSource.adwordsClickInfo.gclId': 'UNK', 
    'trafficSource.adwordsClickInfo.adNetworkType': 'UNK', 
    'trafficSource.adContent': 'UNK',
})

In [0]:
df['totals.pageviews'] = df[['totals.pageviews']].fillna(df['totals.pageviews'].mean())

In [0]:
df[df['totals.pageviews'] == 3.697734703196941]

,channelGrouping,date,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime,new.returningCustomer
7059,Organic Search,20171130,Chrome,desktop,False,Windows,8.515300e+18,Melbourne,Oceania,Australia,...,(not set),True,(not provided),organic,UNK,google,1512040459,2,1512040459,1
28040,Social,20171111,Opera,desktop,False,Windows,6.043600e+18,not available in demo dataset,Europe,Russia,...,(not set),False,UNK,referral,/intl/ru/yt/about/,youtube.com,1510432104,1,1510432104,0
28372,Referral,20171111,Chrome,desktop,False,Windows,5.558472e+18,not available in demo dataset,Europe,Italy,...,(not set),True,UNK,referral,/,it.search.yahoo.com,1510426542,2,1510426542,1
36174,Social,20161112,Chrome,desktop,False,Windows,1.854441e+18,not available in demo dataset,Americas,Argentina,...,(not set),True,UNK,referral,/yt/about/es/,youtube.com,1478952119,7,1478952119,1
36179,Social,20161112,Chrome,desktop,False,Windows,1.854441e+18,not available in demo dataset,Americas,Argentina,...,(not set),True,UNK,referral,/yt/about/es/,youtube.com,1478940943,4,1478940943,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500466,Referral,20161019,Chrome,desktop,False,Macintosh,5.617770e+18,Mountain View,Americas,United States,...,(not set),True,UNK,(none),/,(direct),1476907903,4,1476907903,1
1501211,Paid Search,20170319,Firefox,desktop,False,Windows,7.417511e+18,not available in demo dataset,Asia,Nepal,...,(not set),True,(Remarketing/Content targeting),cpc,UNK,google,1489918339,3,1489918339,1
1505408,Organic Search,20160907,Chrome,desktop,False,Windows,5.784901e+18,Hong Kong,Asia,Hong Kong,...,(not set),True,(not provided),organic,UNK,google,1473244529,4,1473244529,1
1517003,Social,20180216,Mozilla Compatible Agent,desktop,False,(not set),9.515185e+18,not available in demo dataset,Europe,Austria,...,(not set),True,UNK,referral,/intl/de/yt/advertise/,youtube.com,1518805563,3,1518805563,1


In [0]:
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing

In [0]:
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

In [0]:
y = df['new.returningCustomer']
X = df.drop(columns=['new.returningCustomer'])

In [0]:
X.dtypes

channelGrouping                                       object
date                                                   int64
device.browser                                        object
device.deviceCategory                                 object
device.isMobile                                         bool
device.operatingSystem                                object
fullVisitorId                                        float64
geoNetwork.city                                       object
geoNetwork.continent                                  object
geoNetwork.country                                    object
geoNetwork.networkDomain                              object
geoNetwork.region                                     object
geoNetwork.subContinent                               object
socialEngagementType                                  object
totals.bounces                                       float64
totals.hits                                            int64
totals.newVisits        

In [0]:
lb = preprocessing.LabelBinarizer()
X['trafficSource.isTrueDirect'] = lb.fit_transform(X['trafficSource.isTrueDirect'])
X['device.isMobile'] = lb.transform(X['device.isMobile'])

In [0]:
X['visitStartTime'].dtype == np.int64

ignoreColumns = {''}
for column in X.columns:
    if (X[column].dtype != np.int64 and X[column].dtype != np.float64):
        print(f"Converting column: {column}")

        le = preprocessing.LabelEncoder()
        X[column] = le.fit_transform(X[column])
        del le

Converting column: channelGrouping
Converting column: device.browser
Converting column: device.deviceCategory
Converting column: device.operatingSystem
Converting column: geoNetwork.city
Converting column: geoNetwork.continent
Converting column: geoNetwork.country
Converting column: geoNetwork.networkDomain
Converting column: geoNetwork.region
Converting column: geoNetwork.subContinent
Converting column: socialEngagementType
Converting column: trafficSource.adContent
Converting column: trafficSource.adwordsClickInfo.adNetworkType
Converting column: trafficSource.adwordsClickInfo.criteriaParameters
Converting column: trafficSource.adwordsClickInfo.gclId
Converting column: trafficSource.adwordsClickInfo.slot
Converting column: trafficSource.campaign
Converting column: trafficSource.keyword
Converting column: trafficSource.medium
Converting column: trafficSource.referralPath
Converting column: trafficSource.source


In [0]:
X.dtypes

channelGrouping                                        int64
date                                                   int64
device.browser                                         int64
device.deviceCategory                                  int64
device.isMobile                                        int64
device.operatingSystem                                 int64
fullVisitorId                                        float64
geoNetwork.city                                        int64
geoNetwork.continent                                   int64
geoNetwork.country                                     int64
geoNetwork.networkDomain                               int64
geoNetwork.region                                      int64
geoNetwork.subContinent                                int64
socialEngagementType                                   int64
totals.bounces                                       float64
totals.hits                                            int64
totals.newVisits        

In [0]:
forest.fit(X, y)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=250,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [0]:
importances = forest.feature_importances_

In [0]:
indices = np.argsort(importances)[::-1]
indices

array([16, 31,  6, 38, 36,  1, 10, 19, 15, 17,  7, 34, 35, 33,  9, 11,  0,
        5, 37,  2, 12, 18,  8, 14, 27, 32,  3,  4, 24, 25, 30, 29, 21, 20,
       22, 28, 26, 23, 13])

In [0]:
drop_cols = []

for i in range(len(indices)):
    print(f"{X.columns[indices[i]]}:\t {importances[indices[i]]}")
    if importances[indices[i]] == 0:
        drop_cols += [X.columns[indices[i]]]
        
drop_cols

totals.newVisits:	 0.38450705412032943
trafficSource.isTrueDirect:	 0.10948415285880933
fullVisitorId:	 0.05611508722152668
visitStartTime:	 0.05425620218629295
visitId:	 0.05423257723774492
date:	 0.042025202344342046
geoNetwork.networkDomain:	 0.03280811477784845
totals.timeOnSite:	 0.030494312549106686
totals.hits:	 0.02034284081860052
totals.pageviews:	 0.01863699084668348
geoNetwork.city:	 0.017835717865824923
trafficSource.referralPath:	 0.017056887765920382
trafficSource.source:	 0.017019809442060867
trafficSource.medium:	 0.01585853571814412
geoNetwork.country:	 0.014905625729085197
geoNetwork.region:	 0.014701206604422177
channelGrouping:	 0.013901539648458241
device.operatingSystem:	 0.012297899192513916
visitNumber:	 0.011237798898248535
device.browser:	 0.010480352854414125
geoNetwork.subContinent:	 0.009885255470939042
totals.sessionQualityDim:	 0.009145731056784958
geoNetwork.continent:	 0.007346562596670738
totals.bounces:	 0.00538243391437102
trafficSource.adwordsClickI

['trafficSource.adwordsClickInfo.criteriaParameters',
 'totals.visits',
 'socialEngagementType']

In [0]:
drop_cols = ['trafficSource.adwordsClickInfo.criteriaParameters', 'totals.visits', 'socialEngagementType']



In [0]:
df = df.drop(columns=drop_cols)

In [0]:
df.isna().sum(axis=0)

channelGrouping                                      0
date                                                 0
device.browser                                       0
device.deviceCategory                                0
device.isMobile                                      0
device.operatingSystem                               0
fullVisitorId                                        0
geoNetwork.city                                      0
geoNetwork.continent                                 0
geoNetwork.country                                   0
geoNetwork.networkDomain                             0
geoNetwork.region                                    0
geoNetwork.subContinent                              0
socialEngagementType                                 0
totals.bounces                                       0
totals.hits                                          0
totals.newVisits                                     0
totals.pageviews                                     0
totals.ses